# Codigo para geração do indice invertido

In [50]:
import pandas
import math
import nltk
import numpy as np

In [2]:
class WordInfo:
    'Informação sobre a palavra'

    def __init__(self, word) :
        self.word = word
        self.idf = 0
        self.docs = {}

    def found(self, doc_id) :
        if(doc_id in self.docs) :
            self.docs[doc_id] += 1
        else :
            self.docs[doc_id] = 1
            
    def calculateIDF(self, totaldocs) :
        df = len(self.docs)
        if (df > 0) :
            self.idf = math.log((totaldocs + 1)/df)
            
    def getIds(self) :
        return list(self.docs.keys())

In [19]:
tabela = pandas.read_csv('estadao_noticias_eleicao.csv')
tabela.fillna('', inplace=True)

In [4]:
mapa = {} # Dicionario/Mapa que representa o indice invertido

In [36]:
documentos = {}

In [37]:
for index, linha in tabela.iterrows() : # Iterando sobre as noticias no arquivo
    id = linha['idNoticia'] # Recuperando o idNoticia da noticia atual
    texto = nltk.word_tokenize(linha['titulo'] + ' ' + linha['subTitulo'] + ' ' + linha['conteudo'])
    documentos[id] = len(texto) - 2
    for palavra in texto: # Iterando sobre as palavras na Noticia
        if (palavra.isalpha()):
            if (palavra.lower() not in mapa) : 
                mapa[palavra.lower()] = WordInfo(palavra.lower())
            mapa[palavra.lower()].found(id)

for k in mapa.keys() :
    mapa[k].calculateIDF(len(documentos))

# Funções de Consultas Booleanas(And, Or e geral)

In [43]:
def searchAnd(palavra1, palavra2) :
    docs1 = mapa[palavra1.lower()].getIds() # Recupera todos as noticias em que "palavra1" ocorreu
    docs2 = mapa[palavra2.lower()].getIds() # Recupera todos as noticias em que "palavra2" ocorreu
    result = set() # Cria um conjunto vazio
    result.update(docs1) # Preenche o conjunto com os ids das noticias que "palavra1" aparece
    result = result.intersection(docs2) # Mantem no conjunto apenas os ids que as duas palavras ocorrem.
    return list(result)

In [44]:
def searchOr(palavra1, palavra2) :
    docs1 = mapa[palavra1.lower()].getIds() # Recupera todos as noticias em que "palavra1" ocorreu
    docs2 = mapa[palavra2.lower()].getIds() # Recupera todos as noticias em que "palavra2" ocorreu
    result = set() # Cria um conjunto vazio
    result.update(docs1) # Preenche o conjunto com os ids das noticias que "palavra1" aparece
    result.update(docs2) # Preenche o conjunto com os ids das noticias que "palavra2" aparece, por ser conjunto as duplicadas são eliminadas
    return list(result)

In [45]:
def search(consulta) :
    partes = consulta.split(' ')
    if (len(partes) < 2) : # Se a consulta só tem uma palavra
        return mapa[partes[0].lower()].getIds() # Recupera os ids que a palavra aparece
    elif (partes[1].upper() == 'AND') : # Se é uma consulta AND
        return searchAnd(partes[0], partes[2]) # Chama a função de consulta AND
    elif (partes[1].upper() == 'OR') : # Se é uma consulta OR
        return searchOr(partes[0], partes[2]) # Chama a função de consulta OR

# Funções de Consultas Vetoriais

In [45]:
def busca_binaria(consulta) :
    palavras = consulta.split(' ')
    relevant_docs = set(mapa[palavras[0]].getIds())
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    return list(relevant_docs)[:5]

# Asserts Consultas Booleanas

## 1. debate, presidenciável (AND e OR)

In [46]:
assert len(search("debate OR presidencial")) == 1770

In [47]:
assert len(search("debate AND presidencial")) == 201

# 2. presidenciáveis, corruptos (AND e OR)

In [48]:
assert len(search("presidenciáveis OR corruptos")) == 164

In [53]:
assert len(search("presidenciáveis AND corruptos")) == 0

# 3. Belo, Horizonte (AND e OR)

In [50]:
assert len(search("Belo OR Horizonte")) == 331

In [51]:
assert len(search("Belo AND Horizonte")) == 242

# Avaliação Buscas Vetoriais

In [48]:
gabarito = pandas.read_csv('gabarito.csv')

In [51]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## 1. Mapk Busca Binaria

In [ ]:
#mapk(gabarito.busca_binaria,)

## 2. Mapk Busca TF

## 3. Mapk TF-IDF

## 4. Mapk BM25